<h1> This notebook is expected to analyse trends and oportunities in global markets </h1>

<h2>First we get an overview of historical growth by sector and exchange </h2>

In [ ]:
import sys
import pandas as pd
sys.path.append("../getData")
sys.path.append("../functions")
import bdStocks
import numpy as np
import transformationsDataframes as tD
bd=bdStocks.getData()
CARGAR_NUEVOS_DATOS=False
sql="select distinct(sector) from sectors"
sql1="select distinct(exchange) from stocks"
sectors=[e[0] for e in bd. executeQuery(sql)]
exchanges=[e[0] for e in bd. executeQuery(sql1)]
exchanges=["LSE","MC","XETRA","PA","US","TO","MI"]
exchangesIndex={"BE":"DAX30","NEO":"SPTSX","LSE":"FTSE100","MC":"IBEX35","US":"SP500","XETRA":"DAX30","VX":"SMI","PA":"CAC40","TO":"SPTSX","MI":"FTSEMIB","AS":"AEX"}
columnasPrecios=["Adjusted_close","sector"]
columnasFundamental=["netIncome","ebitda","totalRevenue","totalAssets","sector"]

In [ ]:
bd=bdStocks.getData()
bd.getDataByExchange(tipo="fundamental",exchange="MC",sector="Industrials",columnas=["netIncome"]).tail()

In [ ]:
def analyseData(exchange,sector,preciosIndice):
    fundamental=bd.getDataByExchange(tipo="fundamental",exchange=exchange,sector=sector,columnas=columnasFundamental)
    fundamental=tD.pasarAtrimestres(fundamental)
    precios=bd.getDataByExchange(tipo="precios",exchange=exchange,sector=sector,columnas=columnasPrecios)
    precios=tD.pasarAMensual(precios)
    #print(precios.loc[precios.index.get_level_values(1)==precios.index.get_level_values(1)[0]].tail(5))
    #print(fundamental.loc[fundamental.index.get_level_values(1)==fundamental.index.get_level_values(1)[0]].tail(5))
    agrupados=precios.groupby(["stock"])
    precios["stock"]=precios.index.get_level_values(1)
    medias=agrupados.mean()
   
    des=agrupados.std()
    precios=precios.apply(lambda x:(x["Adjusted_close"]-medias.loc[x["stock"]])/des.loc[x["stock"]],axis=1)
    print(precios.tail())
    precios=precios.groupby(level=0).mean()
    print(precios.tail())
    return precios,precios["Adjusted_close"].corr(preciosIndice["Close"])

In [ ]:
if CARGAR_NUEVOS_DATOS:
    diccionario={}
    dataframeSectores=None
    for exchange in exchanges:
        sectors=bd. getJustSectors(exchange)
        preciosIndice=bd.getIndexPrizes(exchangesIndex[exchange])
        preciosIndice= tD.pasarAMensual(preciosIndice)
        print(preciosIndice.tail(5))
        for sector in sectors:

            if sector is not None and sector!='':
                try:
                    print(exchange+"_"+sector)
                    data1,diccionario[exchange+"_"+sector]=analyseData(exchange,sector,preciosIndice)
                    data1["sector"]=sector
                    data1["exchange"]=exchange
                    data1.set_index(["exchange","sector"],inplace=True,append=True)
                    if data1 is None:
                        dataframeSectores=data1
                    else:
                        dataframeSectores=pd.concat([dataframeSectores,data1])

                except Exception as e:
                    print(e)

In [ ]:
dataframeSectores=bd.getPreciosSectores()

In [ ]:
if CARGAR_NUEVOS_DATOS:
    import sqlalchemy
    engine = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}?charset=utf8'.format(
        "usuario1", "password", "localhost:33062", "stocks"), pool_recycle=3600, pool_size=5).connect()
    for exchange in exchanges:
        df=dataframeSectores.loc[dataframeSectores.index.get_level_values(1)==exchange]
        df=df.rename(columns={"Adjusted_close":"precio"})
        print(df.tail())
        df.to_sql("preciosPorSectores", engine,
                       if_exists="append", chunksize=1000)

In [ ]:
def analysePrizes():
    fundamental=bd.getDataByExchange(tipo="fundamental",exchange=exchange,sector=sector,columnas=columnasFundamental)
    fundamental=tD.pasarAtrimestres(fundamental)
    precios=bd.getDataByExchange(tipo="precios",exchange=exchange,sector=sector,columnas=columnasPrecios)
    precios=tD.pasarAMensual(precios)
    precios=precios.loc[:,~precios.columns.duplicated()]
    
    agrupados=precios.groupby(["stock"])
    
    medias=agrupados.mean()
    
    des=agrupados.std()
    precios=precios.apply(lambda x:(x["Adjusted_close"]-medias.loc[x["stock"]])/des.loc[x["stock"]],axis=1)
    precios=precios.groupby(level=0).mean()
    return precios["Adjusted_close"].corr(preciosIndice["Close"])

In [ ]:
correlaciones={}
for exchange in exchanges:
    df1=dataframeSectores.loc[exchange,slice(None),slice(None),:].droplevel(0)
    df1=df1.unstack(level=0, fill_value=None)
    correlaciones[exchange]=df1.corr().droplevel(0).droplevel(0,axis=1)

In [ ]:
import matplotlib.pyplot as plt
exchange="LSE"
NUMERO_STOCKS_POR_SECTOR=("select sector ,count(*) as cuenta from sectors where exchange=%s  group by (sector);",(exchange,))
a=bd. executeQuery(NUMERO_STOCKS_POR_SECTOR[0],params=NUMERO_STOCKS_POR_SECTOR[1])
data=pd.DataFrame(a)
data.columns=["sector","cuenta"]
SECTORES=[e for e in (data.loc[data.cuenta>20].sector) if e is not None and e!='']
correlaciones[exchange]=correlaciones[exchange].loc[SECTORES,SECTORES]
plt.pcolormesh(np.arange(len(correlaciones[exchange].columns)+1), np.arange(len(correlaciones[exchange].columns)+1), correlaciones[exchange].values, cmap = 'Blues')
plt.xticks(np.arange(len(correlaciones[exchange].columns)),correlaciones[exchange].columns, rotation = 'vertical')
plt.yticks(np.arange(len(correlaciones[exchange].columns)), correlaciones[exchange].columns, rotation = 'horizontal')
print(correlaciones[exchange])

In [ ]:
round(correlaciones[exchange],2)

In [ ]:
round(correlaciones[exchange],3)
# en MC las menos correladas son HealthCare,Energy (hay 2),Comunication Services, Technology y Real Estate
# en US las menos correladas son Auto Components,Energy, Basic Materials,Technology
# en XETRA son Real State, Utilities, Technology,Coumunication Services
# en CA  Energy, Utilities, Technology, Comunication and Services
# en LSE HeathCare y Technology

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
dataframeSectores=dataframeSectores.rename(columns={"Adjusted_close":"precio"})
#dataframeSectores1=dataframeSectores[dataframeSectores.index.get_level_values(1).isin(["Technology"])]
sectors=["Basic Materials","Consumer Cyclical","Technology","Healthcare","Real Estate","Communication Services","Utilities"]
#u=df_aux.groupby(level=["exchange","sector"])
exs=["US","TO"]
df_aux=dataframeSectores.loc[(dataframeSectores.index.get_level_values(0).isin(exs))&(dataframeSectores.index.get_level_values(2)>dt.datetime(2010,1,1))]
for exchange in exs:
    NUMERO_STOCKS_POR_SECTOR=("select sector ,count(*) as cuenta from sectors where exchange=%s  group by (sector);",(exchange,))
    a=bd. executeQuery(NUMERO_STOCKS_POR_SECTOR[0],params=NUMERO_STOCKS_POR_SECTOR[1])
    fig=plt.figure(figsize=(15,15))
    data=pd.DataFrame(a)
    data.columns=["sector","cuenta"]
    plt.pie(data["cuenta"],labels=data["sector"],autopct='%1.1f%%')
    plt.title(exchange)
    print(data)
for sector in sectors:
   

    df=df_aux.loc[df_aux.index.get_level_values(1)==sector]
    sns.relplot(x =df.index.get_level_values(2)\
                , y = 'precio', data = df, height=10,kind="line", hue=df.index.get_level_values(0))
   
    plt.title(sector)
    
  


In [ ]:
dataframeSectores.tail()

In [ ]:
exchanges=iter(exchanges)

In [ ]:
exchange=next(exchanges)
sectors=bd. getJustSectors(exchange)
preciosIndice=bd.getIndexPrizes(exchangesIndex[exchange])
preciosIndice= tD.pasarAMensual(preciosIndice)
sectors=iter(sectors)       
        

In [ ]:
sector=next(sectors)

In [ ]:
exchange,sector

In [ ]:
fundamental=bd.getDataByExchange(tipo="fundamental",exchange=exchange,sector=sector,columnas=columnasFundamental)
fundamental=fundamental.loc[:,~fundamental.columns.duplicated()]
fundamental=tD.pasarAtrimestres(fundamental,strings=True)
print(fundamental.tail())
precios=bd.getDataByExchange(tipo="precios",exchange=exchange,sector=sector,columnas=columnasPrecios)
precios=tD.pasarAMensual(precios)
print(precios.loc[precios.index.get_value(1)=="TEF"].tail())
precios=precios.loc[:,~precios.columns.duplicated()]
agrupados=precios.groupby(["stock"])
medias=agrupados.mean()
des=agrupados.std()
preciosScaled=precios.copy()
preciosScaled["Adjusted_close"]=preciosScaled.apply(lambda x:(x["Adjusted_close"]-medias.loc[x["stock"]]/des.loc[x["stock"]]),axis=1)


In [ ]:
precios.loc[precios["stock"]=="TEF"].sort_index(ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
preciosScaled.tail()
ciudades=preciosScaled["stock"].value_counts()
fig,ax=plt.subplots(figsize=(15,7))
ax.pie(ciudades,labels=ciudades.index)
fundamental.shape,fundamental.isna().sum(),precios.shape,precios.isna().sum()

In [ ]:
stocks=set(fundamental["stock"])
descriptions=bd.getDescriptions(exchange)
for stock in stocks:
    print(descriptions[stock])
    print("----------------")

In [ ]:
datas=[]
from functools import reduce
for stock in stocks:
    data=preciosScaled.loc[preciosScaled["stock"]==stock,["Adjusted_close"]]


    data.rename(columns={"Adjusted_close":stock},inplace=True)
    datas.append(data)
    if(len(datas)>0):
            df = reduce(
                    lambda left,right: pd.merge(
                        left,
                        right,
                        left_index=True, 
                        right_index=True, 
                        how='outer',
                        suffixes=('', '_drop')
                    ), 
                    datas
                )

In [ ]:
preciosScaled.loc[preciosScaled["stock"]=="TEF"].tail()